In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Two Grid Correction

Before we do multigrid, we can understand the basic flow by using 2 grids: a fine and coarse grid.

We want to solve:

$$\nabla^2 \phi = f$$

with suitable boundary conditions.  Imagine that we have a approximate solution, $\tilde{\phi}$, then we can define the error with respect to the true solution, $\phi^{\mathrm{true}}$ as:

$$e = \phi^{\mathrm{true}} - \tilde{\phi}$$

Our Poisson equation is linear, so we can apply the $\nabla^2$ operator to our error:

$$\nabla^2 e = \nabla^2 \phi^{\mathrm{true}} - \nabla^2 \tilde{\phi} = f - \nabla^2 \tilde{\phi}$$

This since $\tilde{\phi}$ is just our approximation to $\phi^{\mathrm{true}}$, this is just the residual, so:

$$\nabla^2 e = r$$

Notice that this is the same type of equation as our original equation.  But the boundary conditions are now homogeneous (of the same type as the original) regardless of whether they were inhomogeneous or homogeneous originally.  For example:

* If the BCs were inhomogenous Dirichlet, $\phi(a) = A$, then 

  $$e(a) = \phi^{\mathrm{true}}(a) - \tilde{\phi}(a) = 0$$
  
  since both the true solution and our approximation satisfy the same boundary conditions.
  
* Likewise, if the BCs were inhomogenous Neumann, $\phi^\prime(a) = C$, then

  $$e^{\prime}(a) = \phi^{\mathrm{true},\prime}(a) - \tilde{\phi}^{\prime}(a) = 0$$

The key idea in multigrid is that long wavelength errors take longer to smooth away.  But long wavelength errors look shorter wavelength on a coarser grid.  This suggests that we smooth a bit on a fine grid to get rid of the short wavelength errors and then transfer down to a coarser grid, and smooth the error equation there, and then bring the error back to the fine grid to correct the fine grid solution.

We'll use the superscript $h$ to represent the fine grid and the superscript $2h$ to represent a grid that is coarser by a factor of 2.

Here's the basic flow:

<div style="border: solid; padding: 10px; width: 80%; margin: 0 auto; background: #dddddd">
    
* Smooth for $N_\mathrm{smooth}$ iterations on the fine grid, solving $\nabla^2 \phi = f$ to get an approximation $\tilde{\phi}$

* Compute the residual, on the fine grid:

  $$r^h = f - \nabla^2 \tilde{\phi}$$
  
* Restrict $r^h$ to the coarse grid:

  $$r^{2h} \leftarrow r^h$$
  
* Solve the error equation on the coarse grid:

  $$\nabla^2 e^{2h} = r^{2h}$$
  
* Prolong the error from the coarse grid up to the fine grid:

  $$e^{2h} \rightarrow e^{h}$$
  
* Correct the fine grid solution, since by the definition of $e$, $\phi^{\mathrm{true}} = \tilde{\phi} + e$

  $$\tilde{\phi}^h \leftarrow \tilde{\phi}^h + e^h$$
  
* This correction may have introduced some short wavelength noise (from the prolongation), so smooth $\nabla^2 \phi = f$ on the fine grid for $N_\mathrm{smooth}$ iterations.
</div>

For now, to solve the error equation, we'll resort to simply smoothing until all of residual is below our desired tolerance.  In a little bit though, we'll see that we can just do this procedure recursively for that step.